In [1]:
import cv2 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from keras.models import Sequential 
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Dense,Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split 
from keras.preprocessing.image import ImageDataGenerator 
import pickle 
import os 
import random 
print(cv2.__version__)


2022-06-16 11:33:19.885073: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/monkey/.local/lib/python3.9/site-packages/cv2/../../lib64:
2022-06-16 11:33:19.885141: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


4.5.5


In [2]:
path = "./data"
images = [] 
class_no = [] 
no_of_classes = len(os.listdir(path))
for i in os.listdir(path):
    for file_name in os.listdir(path+"/"+i):
        image = cv2.imread(path+"/"+i+"/"+file_name)
        images.append(image)
        class_no.append(i)
    break
images = np.array(images)
class_no = np.array(class_no)


In [3]:
test_ratio = 0.2 
validation_ratio = 0.2 
x_train,x_test,y_train,y_test = train_test_split(images,class_no,test_size=test_ratio)
x_train,x_validation,y_train,y_validation = train_test_split(x_train,y_train,test_size=validation_ratio)


In [4]:
labels_dataframe = pd.read_csv("labels.csv")
labels_dataframe.head()

,ClassId,Name
0,0,Speed limit (20km/h)
1,1,Speed limit (30km/h)
2,2,Speed limit (50km/h)
3,3,Speed limit (60km/h)
4,4,Speed limit (70km/h)


In [5]:
def grayscale(image):
    return cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
def equalize(image):
    return cv2.equalizeHist(image)
def preprocessing_image(image):
    image = equalize(grayscale(image))
    image = image / 255 
    return image


In [6]:
x_train = np.array(list(map(preprocessing_image,x_train)))
x_test = np.array(list(map(preprocessing_image,x_test)))
x_validation = np.array(list(map(preprocessing_image,x_validation)))
shape = x_train.shape
x_train = x_train.reshape(shape[0],shape[1],shape[2],1)
shape = x_test.shape
x_test = x_test.reshape(shape[0],shape[1],shape[2],1)
shape = x_validation.shape
x_validation = x_validation.reshape(shape[0],shape[1],shape[2],1)

In [7]:
image_generator = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,
                                zoom_range=0.2,shear_range=0.1,rotation_range=10)

In [8]:
image_generator.fit(x_train)


In [9]:
batches = image_generator.flow(x_train,y_train,batch_size=20)
x_batch, y_batch = next(batches)

In [ ]:
no_of_classes = 134 
print(len(y_train))
y_train =  to_categorical(np.array(y_train),no_of_classes)
# y_test =  to_categorical(y_test,no_of_classes)
# y_validation = to_categorical(y_validation,no_of_classes)


In [10]:
def build_model():
    
    filters = 60
    size_five = (5,5)
    size_three = (3,3)
    size_two = (2,2)
    nodes = 500
    model = Sequential()
    model.add((Conv2D(filters,size_five,input_shape=(32,32,1),activation="relu")))
    model.add((Conv2D(filters,size_five,activation="relu")))
    model.add((MaxPooling2D(pool_size=size_two)))
    
    model.add((Conv2D(filters//2,size_three,activation="relu")))
    model.add((Conv2D(filters//2,size_three,activation="relu")))
    model.add((MaxPooling2D(pool_size=size_two)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(nodes,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(no_of_classes,activation="softmax"))
    model.compile(Adam(lr=0.001),loss="categorical_crossentropy",metrics=["accuracy"])
    
    return model 
    
model = build_model()
print(model.summary())
model.fit_generator(image_generator.flow(x_train,y_train,batch_size=50),steps_per_epoch=2000,epochs=30,validation_data=(x_validation,y_validation))

score = model.evaluate(x_test,y_test,verbose=0)

print("Score",score[0])
print("Accuracy",score[1])

print("Saving Model.....")
pickle_out = open("road_classifier_model.p","wb")
pickle.dump(model,pickle_out)
pickle_out.close()










    

2022-06-16 11:33:51.955485: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/monkey/.local/lib/python3.9/site-packages/cv2/../../lib64:
2022-06-16 11:33:51.955555: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-16 11:33:51.955614: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kali): /proc/driver/nvidia/version does not exist
2022-06-16 11:33:51.956242: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 60)        1560      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 60)        90060     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 60)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 30)        16230     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 30)          8130      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 30)         0         
 2D)                                                    

/home/monkey/.local/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
capture = cv2.VideoCapture(0)
model = pickle.load(open("road_classifier_model.p","rb"))
while True :
    _, image = capture.read()
    image = np.array(image)
    image = cv2.resize(image,(32,32))
    image = preprocessing(image)
    image = image.rehape(1,32,32,1)
    prediction = model.predict(image)
    class_index = model.predict_classes(image)
    print("Sign is :")
    print(get_class_name(class_index))
    if cv2.waitKey(1)&0xFF == ord("s"):
        break
